In [4]:
import torch
import pandas as pd
from pathlib import Path
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from random import sample
import os
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import math
import torch.nn as nn


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda


In [6]:
model_path = 'embeding_models/embeding_v1_16_dim.pt'
model = torch.jit.load(model_path).encoder

In [7]:
BASE_DATE = datetime.strptime("2025-01-03", '%Y-%m-%d').timestamp()
print(BASE_DATE)

1735858800.0


In [8]:
tracks_raw_data = pd.read_json(Path('../data_v2/tracks_artists.jsonl'), lines=True)


# NORMALIZE DATES
tracks_raw_data["release_date"] = (pd.to_datetime(tracks_raw_data["release_date"], format='mixed').apply(lambda x: x.timestamp())).div(BASE_DATE)

# NORMALIZE DURATION
tracks_raw_data["duration_ms"] = tracks_raw_data["duration_ms"].div(tracks_raw_data["duration_ms"].max())

# NORMALIZE TEMPO
tracks_raw_data["tempo"] = tracks_raw_data["tempo"].div(tracks_raw_data["tempo"].max())

# EXPLICITE ENCODING
tracks_raw_data["explicit"] = tracks_raw_data["explicit"].apply(lambda x: [0, 1] if x else [1, 0])

# processs ARTIST HASH

def postprocess_hash_to_list(x):
    str_x = str(x)
    if len(str_x) < 8:
        str_x = "0" * (8 - len(str_x)) + str_x
    return [int(x) for x in str_x]

tracks_raw_data["id_artist_hash"] = tracks_raw_data["id_artist_hash"].apply(postprocess_hash_to_list)



In [9]:
class TracksDataset(Dataset):
    def __init__(self, tracks_data: pd.DataFrame):
        self.data = tracks_data

    def __len__(self):
        return len(self.data)

    def get_item(self, idx):
        return self.data.iloc[idx].values

    def __getitem__(self, idx):
        unpacked_data = []
        for data in self.data.iloc[idx].drop("id_track").values:
            if type(data) != list:
                unpacked_data.append(data)
            else:
                unpacked_data += data
        return torch.Tensor(unpacked_data)

In [10]:
embedings_per_id = pd.DataFrame()
embedings_per_id["id_track"] = tracks_raw_data["id_track"]
embedings_per_id["embeding"] = [model(torch.Tensor(x)).detach().cpu().numpy() for x in TracksDataset(tracks_raw_data)]

In [11]:
test_intigers = [687, 852, 528, 562, 426, 1092, 171, 250, 223, 265, 981, 607, 738, 1020, 510, 899, 596, 1047, 826, 669, 923, 905, 1063, 139, 1031]

In [12]:
files_to_load = [file for file in os.listdir("../data_v2/sessions") if int(file.split(".")[0].split("_")[-1]) not in test_intigers]

raw_sessions_data = pd.concat([pd.read_json(Path(f'../data_v2/sessions/{file}'), lines=True) for file in files_to_load])
print(raw_sessions_data.head())

   session_id               timestamp  user_id                track_id  \
0       89426 2024-09-17 10:51:23.000      260  1xPec5BN0Zxv77zrWKq43S   
1       89427 2023-05-20 00:35:36.227      260  7s0lDK7y3XLmI7tcsRAbW0   
2       89427 2023-05-20 00:38:48.054      260  48lQegoLqGAzaRLnMwK0mO   
3       89427 2023-05-20 00:43:30.802      260  48lQegoLqGAzaRLnMwK0mO   
4       89427 2023-05-20 00:44:59.387      260  4usVYcPlxRgRet6YashdCJ   

  event_type  
0       play  
1       play  
2       play  
3       like  
4       play  


In [13]:
merged_sesions_data = raw_sessions_data.merge(embedings_per_id, left_on="track_id", right_on="id_track").drop("id_track", axis=1)
merged_sesions_data["timestamp"] = (pd.to_datetime(merged_sesions_data["timestamp"], format='mixed').apply(lambda x: x.timestamp())).div(BASE_DATE)
merged_sesions_data = pd.get_dummies(merged_sesions_data, columns=["event_type"], dtype = int)

merged_sesions_data = [pd.DataFrame(y) for _, y in merged_sesions_data.groupby('session_id', as_index=False) if len(y) > 1]

NUMBER_OF_INPUTS = 20

In [14]:
class CustomUserSesionsDataset(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def __len__(self):
        return len(self.data)

    def get_item(self, idx):
        return self.data.iloc[idx].values

    def __getitem__(self, idx):
        sessions = []
        for session in self.data[idx].drop("track_id", axis=1).drop("session_id", axis=1).drop("user_id", axis=1).values:
            unpacked_data = np.array([])
            for data in session:
                unpacked_data = np.append(unpacked_data, data)
            sessions.append(torch.tensor(unpacked_data))
        return sessions

In [15]:
user_sesions_dataset = CustomUserSesionsDataset(merged_sesions_data)

In [16]:
COMPARE_PERCENTAGE = 0.25

def pad_sessions_collate(batch, pad_value=0.0):
    input_sequences = [
    ]
    output_sequences = [
    ]

    for batch_element in batch:
        number_of_sequence_data = len(batch_element)
        number_of_reference_data = math.ceil(number_of_sequence_data * COMPARE_PERCENTAGE)

        input_sequences.append(torch.stack(batch_element[:number_of_sequence_data - number_of_reference_data]))
        output_sequences.append(sample(batch_element[number_of_sequence_data - number_of_reference_data:], 1)[0])

    x_lens = [len(x) for x in input_sequences]

    padded_input_sequence = pad_sequence(input_sequences, batch_first=True, padding_value=pad_value)

    return padded_input_sequence, torch.stack([x[1:17] for x in output_sequences]), x_lens


In [17]:
train_loader = DataLoader(user_sesions_dataset, batch_size=8192, shuffle=True, collate_fn=pad_sessions_collate, num_workers=2)

In [18]:
class UsertPreferenceGenerator(torch.nn.Module):
    def __init__(self):
        super(UsertPreferenceGenerator, self).__init__()
        self.num_layers = 1
        self.hidden_size = 128
        self.lstm = torch.nn.LSTM(input_size=NUMBER_OF_INPUTS, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.activation = torch.nn.ReLU()
        self.linear = torch.nn.Linear(self.hidden_size, 16)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, x_lens, hidden):

        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False).to(device)
        packed_output, hidden = self.lstm(x_packed, hidden)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        x = self.linear(self.activation(output))

        return x, hidden

In [19]:
model = UsertPreferenceGenerator().to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.L1Loss()

UsertPreferenceGenerator(
  (lstm): LSTM(20, 128, batch_first=True)
  (activation): ReLU()
  (linear): Linear(in_features=128, out_features=16, bias=True)
)


/home/mikic202/miniconda3/envs/pt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# Training loop
model = UsertPreferenceGenerator().to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.L1Loss()


model.train()
for epoch in range(6):
    loss_sum = 0
    for x, targets, x_lens in train_loader:
        x = x.float().to(device)
        targets = targets.to(torch.float).to(device)

        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)

        preds, _ = model(x, x_lens, (hidden, state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        last_pred = preds[:, -1, :]
        loss = loss_fun(last_pred, targets)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
    print(f"Epoch: {epoch}, loss: {loss_sum:.3}")

UsertPreferenceGenerator(
  (lstm): LSTM(20, 128, batch_first=True)
  (activation): ReLU()
  (linear): Linear(in_features=128, out_features=16, bias=True)
)
Epoch: 0, loss: 2.27e+02
Epoch: 1, loss: 2.25e+02


In [ ]:
import torch.autograd.profiler as profiler

with profiler.profile(use_cuda=True) as prof:
    preds, _ = model(x, x_lens, (hidden, state))
print(prof.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             aten::_pad_packed_sequence        10.52%       1.156ms        32.01%       3.519ms       3.519ms     909.000us         8.04%       3.522ms       3.522ms             1  
                                             aten::sort        12.44%       1.368ms        27.00%       2.968ms       1.484ms       1.377ms        12.18%       2.978ms       1.489ms             2  
         

STAGE:2024-12-24 14:20:35 4443:4443 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-12-24 14:20:35 4443:4443 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-12-24 14:20:35 4443:4443 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [ ]:
model([[0.993923, 4.7692013 ,  0.50016433,  1.8391552 , -4.93348   ,  1.0898505 ,
          1.9289817 , 10.784728  , -3.9150748 ,  5.587146  ,  7.239292  ,
          7.1324625 ,  2.55823   ,  4.176082  ,  8.504707  ,  5.0005746 ,
         -2.202329, 0, 1, 0]], [1], torch.zeros(1, 1, 32))

AttributeError: 'list' object has no attribute 'device'

TypeError: UsertPreferenceGenerator.forward() missing 2 required positional arguments: 'x_lens' and 'hidden'